# **Source & destination image creating**

In [1]:
import cv2
import numpy as np
import os, glob
import time
import json
from tqdm import tqdm
import random

In [2]:
import matplotlib.pyplot as plt 
%matplotlib inline

In [13]:
dataset_size = 31
img_shape = 64
dataset_dir = '/home/admin1/Document/CITYSCAPES_DATASET'   
save_dir = '/home/admin1/Programming/HIWI_MRT/gp-gan_augmentation/GP-GAN/DataBase'    

highBrightness_value = 50
lowBrightness_value = -50

highContrast_value = 30
lowContrast_value = -30

### Creating of data path list with "person" class

In [14]:
def path_creater(json_path):
    data_name = '_'.join(json_path.split('/')[-1].split('_')[:-2])
    mask_dir = '/'.join(json_path.split('/')[:-1])
    img_dir =  mask_dir.replace('gtFine_trainvaltest/gtFine', 'leftImg8bit_trainvaltest/leftImg8bit')
    mask_path = os.path.join(mask_dir, str(data_name) + '_gtFine_labelIds.png')
    img_path = os.path.join(img_dir, str(data_name) + '_leftImg8bit.png')
    
    return img_path, mask_path

In [15]:
def path_list_creater():
    data_counter = 0
    class_data_counter = 0

    class_data_list = []
    json_paths = glob.glob(os.path.join(dataset_dir,'gtFine_trainvaltest','gtFine','*','*','*.json'))

    for json_path in tqdm(json_paths, total=len(json_paths)):  
        with open(json_path) as json_file:
            json_data = json.load(json_file) 
            obj_key = 'objects'
            if obj_key in  json_data:
                class_list =[label['label'] for label in json_data[obj_key]]
                if 'person' in class_list:
                    img_path, mask_path = path_creater(json_path)
                    act_data = [img_path, mask_path]
                    class_data_list.append(act_data)
                    class_data_counter += 1

        data_counter += 1

    print('%s images with class "Person" in the dataset' % (class_data_counter))
    print("The cityscapes dataset consists of %s images" % (data_counter))
    
    return class_data_list

### Dataset creating

In [16]:
def data_name(path): 
    name = '_'.join(path[1].split('/')[-1].split('_')[:-2])
    
    return name

In [17]:
def data_loader(path):    
    img_path = path[0]; mask_path = path[1]
     
    img = cv2.imread(img_path); img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    mask = cv2.imread(mask_path); mask = cv2.cvtColor(mask, cv2.COLOR_BGR2RGB)
    
    return img, mask 

In [18]:
def data_saver(data_name, dst, src, id_data):
    dst = cv2.cvtColor(dst, cv2.COLOR_RGB2BGR)
    src = cv2.cvtColor(src, cv2.COLOR_RGB2BGR)
    
    dst_path = os.path.join(save_dir,'train_data', 'dst', data_name + '_' + str(id_data) + '.jpg')
    src_path = os.path.join(save_dir,'train_data', 'src', data_name + '_' + str(id_data) +'.jpg')
    
    cv2.imwrite(dst_path, dst)
    cv2.imwrite(src_path, src)

In [19]:
# Augmentation
def brightness_contrast(img, hBrightness, lBrightness, hContrast, lContrast):   
    
    brightness = random.randint(lBrightness, hBrightness)
    contrast = random.randint(lContrast, hContrast)
    if brightness != 0:
        if brightness > 0:
            shadow = brightness
            highlight = 255
        else:
            shadow = 0
            highlight = 255 + brightness
        alpha_b = (highlight - shadow) / 255
        gamma_b = shadow

        buf = cv2.addWeighted(img, alpha_b, img, 0, gamma_b)
    else:
        buf = img.copy()

    if contrast != 0:
        f = 131 * (contrast + 127) / (127 * (131 - contrast))
        alpha_c = f
        gamma_c = 127 * (1 - f)

        buf = cv2.addWeighted(buf, alpha_c, buf, 0, gamma_c)

    return buf


def srcImg_creater(img, mask):
    
    thresh_mask = np.where(mask == person_value, 255, 0).astype(np.uint8)
    gray_mask = cv2.cvtColor(thresh_mask, cv2.COLOR_RGB2GRAY)
    
    obj_contours, obj_hierarchy = cv2.findContours(gray_mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    obj_areas = np.array([cv2.contourArea(obj_contour) for obj_contour in obj_contours]).astype(np.int)
    
    if any(obj_areas):
        person_ids = np.where(obj_areas > 0)[0].astype(np.int)

        new_img = img.copy()
        for person_id in person_ids:

            obj_mask = np.full((mask.shape[0], mask.shape[1]), 0, np.uint8)

            obj_mask = cv2.drawContours(obj_mask, obj_contours, contourIdx=person_id, color=255, thickness=-1)
            obj_mask = cv2.cvtColor(obj_mask, cv2.COLOR_GRAY2RGB)

            augment_img = brightness_contrast(img,
                                              hBrightness = highBrightness_value,
                                              lBrightness = lowBrightness_value,
                                              hContrast = highContrast_value,
                                              lContrast = lowContrast_value)
            new_img = np.where((obj_mask == 255), augment_img, new_img)

        return new_img
    else:
        return 'None'

In [20]:
def save_folder_creater(save_dir):
    dst_fold_path = os.path.join(save_dir, 'train_data', 'dst')
    src_fold_path = os.path.join(save_dir, 'train_data', 'src')
    
    if not os.path.isdir(dst_fold_path):
        os.makedirs(dst_fold_path)
    if not os.path.isdir(src_fold_path):
        os.makedirs(src_fold_path)

In [21]:
def data_resizer(Img, Mask, shape):
    w = shape; h = shape
    Img = cv2.resize(Img, (w, h), interpolation=cv2.INTER_AREA)
    Mask = cv2.resize(Mask, (w, h), interpolation=cv2.INTER_NEAREST)  
    
    return Img, Mask

In [22]:
if __name__ == '__main__':
    # https://github.com/mcordts/cityscapesScripts/blob/master/cityscapesscripts/helpers/labels.py
    # |    name    |  id  |
    # _____________________
    # |  'person'  | 24   |
    person_value = 24
    
    id_data = 1
    
    save_folder_creater(save_dir)
    img_paths = path_list_creater()
    
    start_time = time.time()
    while  (id_data <= dataset_size):
        
        #try:
        currentImg_path = random.choice(img_paths)

        # Data name chosing
        imgName = data_name(currentImg_path)

        # Data loading
        dstImg, dstMask = data_loader(currentImg_path)

        # Data resizing
        dstImg, dstMask = data_resizer(dstImg, dstMask, img_shape)

        # Source image creating
        srcImg = srcImg_creater(dstImg, dstMask)
        if srcImg is 'None':
            continue

        # Data resizing
        #dstImg, srcImg = data_resizer(dstImg, srcImg)

        # Data saving 
        data_saver(imgName, dstImg, srcImg, id_data)

        print("------------------- %s rest --------------------" % (dataset_size - id_data))
        id_data += 1
        #except:
        #    print("!!! ERROR   %s   ERROR !!!" % currentImg_path)
        #    continue
        
    print("----------------- %s seconds ----------------" % ( round((time.time() - start_time), 2) ))

100%|██████████| 5000/5000 [00:04<00:00, 1128.83it/s] 


2728 images with class "Person" in the dataset
The cityscapes dataset consists of 5000 images
------------------- 30 rest --------------------
------------------- 29 rest --------------------
------------------- 28 rest --------------------
------------------- 27 rest --------------------
------------------- 26 rest --------------------
------------------- 25 rest --------------------
------------------- 24 rest --------------------
------------------- 23 rest --------------------
------------------- 22 rest --------------------
------------------- 21 rest --------------------
------------------- 20 rest --------------------
------------------- 19 rest --------------------
------------------- 18 rest --------------------
------------------- 17 rest --------------------
------------------- 16 rest --------------------
------------------- 15 rest --------------------
------------------- 14 rest --------------------
------------------- 13 rest --------------------
------------------- 12 r